<div style ='margin: 11px 11px 51px 11px; padding: 55px; border: 1px solid;'>
    <p style='text-align: center; margin: auto'>
    Поставлена задача – получить парсинг книги с веб-сайта.<br>На выходе необходимо получить файл с данными.</p>
</div>

<div style ='margin: 11px 11px 0px 11px; '>
    <p style=''>
        Используем Selenium, а так же BeautyfullSoup для скрейпинга сайта.<br>
    -->Импорт библиотек, составление необходимых функций, входные данные:</p>
</div>


In [1]:
# Библиотеки
import pathlib
from fpdf import FPDF
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from selenium.webdriver.common.keys import Keys
import time
import math
from bs4 import BeautifulSoup
import numpy as np
import selenium.webdriver.common.keys
from selenium.webdriver.common.by import By 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait #  Создаем список со шрифтами 

In [2]:
# убираем предупреждения
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Получаем название книги
def name_book(el):
    el = ' '.join(el.split(' ')[1:-4])
    return el

In [4]:
# Имя автора
def name_author(el):
    el = ' '.join(el.split(' ')[0:-3])
    return el

In [5]:
#  Создаем список со шрифтами 
def listFonts(path):
    list_fonts = []
    currentDirectory = pathlib.Path(path)
    for font in currentDirectory.iterdir():   
        if str(font)[-3:] == 'ttf':
            list_fonts.append(f"{font}")
    return list_fonts[0], list_fonts[1], list_fonts[2] 

In [6]:
# Получаем суп со страницы
def make_soup(driver):
    page_soup = driver.page_source
    soup = BeautifulSoup(page_soup, 'lxml')
    return soup

In [7]:
# Получаем главную страницу книги 
def title(author, book):
    pdf.set_font('bold', '', 12)
    pdf.cell(0, 144, f'{author}', 0, align='C')
    pdf.ln(9)
    pdf.set_font('bold', '', 14)
    pdf.cell(0, 144, f'{book}', 0, align='C') 

##### Определяемся, что будем искать. И указываем ключ поиска 
##### Запускаем драйвер, ищем книги по указанному ключу 
##### Выбираем книгу из списка и собираем в файл (проводим парсинг)

In [8]:
#  что и где ищем
book_name = 'Яблоко'
loveread_ec = 'http://loveread.ec/'

In [9]:
# Получаем список шрифтов 
fonts = listFonts('font')
print(fonts)

('font/Light.ttf', 'font/ThinItalic.ttf', 'font/Regular.ttf')


In [52]:
# Запуск
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(loveread_ec)

WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/table/tbody/tr[2]/td/table/tbody/tr/td[3]/div/form[2]')))

<selenium.webdriver.remote.webelement.WebElement (session="f71f1cc12e250a44b1c5bc0ca7e28af4", element="A99BC8F57687E0955C3496EC777A0D43_element_969")>

In [53]:
what_search = driver.find_element(By.XPATH, '//*[@id="search"]').send_keys(book_name)
btn_search = driver.find_element(By.XPATH, '/html/body/table/tbody/tr[2]/td/table/tbody/tr/td[3]/div/form[2]/input').send_keys(Keys.RETURN)

In [54]:
df_books = pd.DataFrame(columns=['Имя книги', 'Автор'])
fragments = ['Книга', ' - читать онлайн, бесплатно']

In [55]:
listBocks_soup = make_soup(driver)

In [56]:
for el in listBocks_soup.find('ul', class_='let_ul').find_all('li'):
    if el.get('class') != ['contents_bk']:
        book = str(el.contents[1].get('title'))
        author = el.contents[3].get('title')
        new_row = {'Имя книги': book, 'Автор': author} #append row to the dataframe 
        df_books = df_books.append(new_row, ignore_index=True)

In [57]:
df_books['Имя книги'] = df_books['Имя книги'].apply(name_book)
df_books['Автор'] = df_books['Автор'].apply(name_author)

In [58]:
pd.set_option('display.min_rows', 100)
pd.set_option('display.expand_frame_repr', True)

df_books

,Имя книги,Автор
0,Эммануэль. Мадам как яблоко и мед,Эммануэль Арсан
1,Яблоко Монте-Кристо,Дарья Донцова
2,Спелое яблоко раздора,Марина Серова
3,Яблоко раздора,Ирина Щеглова
4,Ведьмино яблоко раздора,Алина Егорова
5,Яблоко Евы,Инга Берристер
6,Яблоко Евы,Анхель де Куатьэ
7,Яблоко Купидона,Ольга Крючкова
8,Райское яблоко,Ирина Муравьева
9,Евино яблоко,Андрей Макаревич


####  Интересует книга Ольги Новиковой. Выбираем её.

In [59]:
index_book = 16

In [60]:
get_href_book = driver.find_element(By.XPATH, f'/html/body/table/tbody/tr[2]/td/table/tbody/tr/td[2]/div[1]/ul[1]/li[{index_book + 2}]/a[1]').get_attribute('href')
driver.get(get_href_book)
link_withuot_npage = driver.find_element(By.XPATH, '/html/body/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr/td/table/tbody/tr[3]/td/p[2]/a').get_attribute('href')[:-1]


<div style ='margin: 50px 11px 0px 11px; '>
    <p style=''>
        Собираем данные о книге.<br></p>
</div>


In [61]:
# Получаем имя автора и количество страниц 
author = driver.find_element(By.XPATH, '/html/body/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr/td/table/tbody/tr[2]/td/p/a[2]/strong').text
driver.get(link_withuot_npage + '1')
count_page = int(driver.find_element(By.XPATH, '//div[@class="navigation"]//a[last()-1]').text)

In [62]:
# шрифты 
light, italic, bold = listFonts('font')

<div style ='margin: 11px 11px 0px 11px; '>
    <p style=''>
    Создаем PDF файл, далее в который записываем все данные.</p>
</div>

In [63]:
# Создаю файл
pdf = FPDF()
pdf.add_page()

In [64]:
# стилизуем
pdf.add_font('light', '', f'{light}', uni=True)
pdf.add_font('bold', '', f'{bold}', uni=True)
pdf.add_font('italic', '', f'{italic}', uni=True)

In [65]:
# Заголовок 
title(author, book_name)

In [66]:
pdf.add_page()

In [67]:
for page in range(1, count_page + 1):
    driver.get(link_withuot_npage + f'{page}')
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '/html/body/table/tbody/tr[2]/td/table/tbody/tr/td[2]')))
    soup = make_soup(driver)
    for el in soup.find('div', class_='MsoNormal').find_all(['div', 'p']): # , recursive=True
        if (el.name == 'div') & (el.get('class') == ['take_h1']): # заголовок 
            pdf.ln(5)
            pdf.set_font('bold', '', 12)
            pdf.multi_cell(0, 4, txt=f'{el.text}',  align="C")
            pdf.ln(3)
        elif el.get('class') == ['MsoNormal']: # основной текст 
            if (el.name == 'p') & (el.find_parents(limit=1)[0].get('class') != ['em']):
                pdf.set_font('light', '', 9)
                pdf.multi_cell(0, 5, txt=f'{el.text}', align='L')
                pdf.set_left_margin(30)
                pdf.set_right_margin(34)
                pdf.ln(2)
            else:
                pdf.set_font('italic', '', 9)
                pdf.multi_cell(0, 4, txt=f'{el.text}', align="L")
        elif (el.get('class') == ['strong']) & (el.name == 'p'):
            if el.text != 'Я есть богатство. Я есть изобилие. Я есть радость':
                pdf.set_font('bold', '', 9)
                pdf.multi_cell(0, 12, txt=f'{el.text}', align='L')
            else:
                pdf.ln(4)
        else:
            continue

In [68]:
pdf.output(f"{book_name}.pdf")

''

In [69]:
driver.quit() 

<div style ='margin: 14px 0; '>
    <p style='font-size: 16px'>
        Закрываем файл, можно его потрогать тут 
<a>https://github.com/AnfisaAnalytics/myProfile/blob/main/files/Яблоко.pdf</a> . Работу драйвера завершим.</p>
</div>

## Задача выполнена